In [2]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
import random
import sys

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, BatchNormalization, Dense, Activation, Flatten
from tensorflow.keras.models import Sequential, Model
#from tensorflow.keras.optimizers import 
from tensorflow.keras.utils import Sequence, to_categorical

# data processing
def get_data(base_path , img_size = (133, 100) , datasize_head = 0 , datasize_end = 1000 , shuffle = True):
    indexes = np.arange(datasize_head , datasize_end)

    if shuffle == True:
        np.random.shuffle(indexes)
    
    images, labels = [], []
    for id_name_actual in indexes:
        image_path = os.path.join(base_path , (str(id_name_actual) + '.jpg'))
        
        # load the image
        image = cv2.imread(image_path , cv2.IMREAD_GRAYSCALE)#Reading Image in GRAY format
        image = cv2.resize(image, img_size)

        if id_name_actual < 3626:
            label = np.array([1 , 0])
        else:
            label = np.array([0 , 1])

        #change data type
        image = image.astype('float32')
        label = label.astype('float32')

        # Normalizing the image and change the type to float
        image = image /255

        images.append(image)
        labels.append(label)

    images = np.array(images)
    labels = np.array(labels)

    images = images.reshape(images.shape[0], img_size[1] ,img_size[0] , 1) 

    return images, labels

# model
pool_size = (2,2)
model = Sequential()
# Normalizes incoming inputs. First layer needs the input shape to work
model.add(BatchNormalization(input_shape=(100,133,1)))
# Conv Layer 1
model.add(Conv2D(4, (3, 3), padding='valid', strides=(1,1), activation = 'relu', name = 'Conv1'))
model.add(Conv2D(4, (3, 3), padding='valid', strides=(1,1), activation = 'relu', name = 'Conv2'))
model.add(MaxPooling2D(pool_size=pool_size))

model.add(Conv2D(8, (3, 3), padding='valid', strides=(1,1), activation = 'relu', name = 'Conv3'))
model.add(Conv2D(8, (3, 3), padding='valid', strides=(1,1), activation = 'relu', name = 'Conv4'))
model.add(MaxPooling2D(pool_size=pool_size))

model.add(Conv2D(16, (3, 3), padding='valid', strides=(1,1), activation = 'relu', name = 'Conv5'))
model.add(Conv2D(16, (3, 3), padding='valid', strides=(1,1), activation = 'relu', name = 'Conv6'))
#model.add(Conv2D(16, (3, 3), padding='valid', strides=(1,1), activation = 'relu', name = 'Conv7'))
model.add(MaxPooling2D(pool_size=pool_size))

model.add(Conv2D(32, (3, 3), padding='valid', strides=(1,1), activation = 'relu', name = 'Conv8'))
model.add(Conv2D(32, (3, 3), padding='valid', strides=(1,1), activation = 'relu', name = 'Conv9'))
#model.add(Conv2D(32, (3, 3), padding='valid', strides=(1,1), activation = 'relu', name = 'Conv10'))
model.add(MaxPooling2D(pool_size=pool_size))

# model.add(Conv2D(64, (3, 3), padding='valid', strides=(1,1), activation = 'relu', name = 'Conv11'))
# model.add(BatchNormalization())
# model.add(Conv2D(64, (3, 3), padding='valid', strides=(1,1), activation = 'relu', name = 'Conv12'))
# model.add(BatchNormalization())
# model.add(Conv2D(64, (3, 3), padding='valid', strides=(1,1), activation = 'relu', name = 'Conv13'))
# model.add(MaxPooling2D(pool_size=pool_size))

#FC1
model.add(Flatten())
model.add(Dense(64))
model.add(BatchNormalization())
model.add(Activation("relu"))

#FC2
model.add(Dense(2))
model.add(Activation("softmax"))

#Define optimizer loss function and merics 
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])


# Training
images , labels = get_data('Lane_detection_dataset' , img_size = (133, 100) , datasize_head = 0 , datasize_end = 7252 , shuffle = True)


# model = Lane_detection_model()
print(model.summary())

print(images.dtype , images.shape)
print(labels.dtype , labels.shape)

# set training
history = model.fit(images,labels,
            validation_split = 0.2,
            batch_size = 100,
            verbose = 1,
            epochs = 6
            )
            
# Save weights of this model
model.save_weights('Lane_detection_model_weight.h5')
model.save('Lane_detection_model.h5')

print(history.history.keys())

# Part 4 - Visualization

# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_2 (Batch (None, 100, 133, 1)       4         
_________________________________________________________________
Conv1 (Conv2D)               (None, 98, 131, 4)        40        
_________________________________________________________________
Conv2 (Conv2D)               (None, 96, 129, 4)        148       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 48, 64, 4)         0         
_________________________________________________________________
Conv3 (Conv2D)               (None, 46, 62, 8)         296       
_________________________________________________________________
Conv4 (Conv2D)               (None, 44, 60, 8)         584       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 22, 30, 8)        

KeyboardInterrupt: 